# Dash

The concept is to create a dashboard that will allow a user to upload a given image of XX by XX pixel. Our pretrained model will then be able to identify the image based on its weather characteristics. 

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
from dash_core_components import Upload
from dash.dependencies import Input, Output, State
from plotly import graph_objs as go
from plotly.graph_objs import *
from PIL import Image
import glob
import base64
import os
import plotly_express as px
import sys
from pathlib import Path
sys.path.append("..")
from src.app import app_tools
from src.data import make_dataset

In [2]:
# !pip install dash-bootstrap-components

# Plotly interactive map onto Dash

In [3]:
styles = {
    'pre': {
        'border': 'grey',
        'overflowX': 'scroll'
    }
}

app.css.append_css({'external_url': 'https://codepen.io/amyoshino/pen/jzXypZ.css'})
# classNames creates the layout for each Div. Banner/two or six columns etc. are essentially varying sizes
    
df = make_dataset.load_raw_labels()
df = df[:100].copy()

df = make_dataset.add_random_lat_lon(df, [-3, -62])
tags = df['tags']

fig = go.Figure(data=go.Scattergeo(
        lon = df['lon'],
        lat = df['lat'],
        mode = 'markers',
        marker = dict(
            size = 4,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
        )))

for tags, data in tags.items():
    fig.add_trace(go.Scatter(
        x=df['lat'],
        y=df['lon'],
        name=tags,
        text=df['tags'],
        ))

fig.update_traces(
    mode='markers',
    marker={'sizemode':'area',
            'sizeref':10})

fig.update_layout(
        title = 'Random lat lon in South America',
        geo = dict(
            scope='south america',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )
    
app = dash.Dash(external_stylesheets=external_stylesheets)

colors={
    'background':'#111111',
    'text':'#7FDBFF'
},

app.layout = html.Div([
    html.Div([
        html.H2(
            children="Amazon Rainforest Satellite Imagery - Booz Allen WiDS Incubator Challenge",
            className='nine columns',
            style={
                'color':'rgb(60,179,113)',
                'height': '75px',
                'lineHeight': '50px',
                'backgroundColor':'rgb(91,91,91)',
                'textAlign': 'left',
                'margin': '10px'
            },
        ),
        ], className='row'),

        html.Div([
            dcc.Markdown('''
            ###### Objectives: 
            (1) Empower the user to understand the landscape of deforestation in the Amazon and adjust imagery collection, (2) Use machine learning and remote sensing data from Landsat satellites to identify regions where deforestation has occurred in remote regions; 
            Label image chips with atmospheric conditions, class cover, and land use.
            This data is from a Kaggle competition called Planet: Understanding the Amazon from Space. [Access the link here.](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/data).
                            '''
                        ),
                        ],
                className='six columns',
                style={
                    'textAlign': 'left',
                    'width':'50%', 'display':'inline-block'
                },
        )

        html.Label(
                children="Atmosheric Conditions",
        )
                style={
                    'textAlign': 'left'
                },
            ),

        html.Div([ 
            dcc.Dropdown(
                id='dropdown1',
                options=[
                    {'label': 'Partly cloudy', 'value': 'partly_cloudy'},
                    {'label': 'Cloudy', 'value': 'cloudy'},
                    {'label': 'Hazy','value': 'haze'},
                    {'label': 'Clear', 'value': 'clear'},
                ],
                    value=['']
                ),
            ],
            style={'width':'50%', 'display':'inline-block'}
        ),

        html.Label(
            children="Land Use",
            style={
                'textAlign': 'left'},
        ),
        html.Div([
            dcc.Dropdown(
            id='dropdown2',
            options=[
                {'label': 'Slash and Burn', 'value': 'slash_and_burn'},
                {'label': 'Selective Logging', 'value': 'selective_logging'},
                {'label': 'Conventional Mining', 'value': 'conventional_mining'},
                {'label': 'Artisinal Mining', 'value': 'artisinal_mining'},
            ],
                value=['']
            ),
        ],
        style={
            'width':'50%', 'display':'inline-block'},
        ),

        html.Label(
            children="Multi-Select Dropdown",
            style={
                'textAlign': 'left'},
        ),

        html.Div([
            dcc.Dropdown(
            id='Multi-Select Dropdown',
                options=[
                    {'label': 'Partly cloudy', 'value': 'partly_cloudy'},
                    {'label': 'Cloudy', 'value': 'cloudy'},
                    {'label': 'Hazy','value': 'haze'},
                    {'label': 'Clear', 'value': 'clear'},
                    {'label': 'Slash and Burn', 'value': 'slash_and_burn'},
                    {'label': 'Selective Logging', 'value': 'selective_logging'},
                    {'label': 'Conventional Mining', 'value': 'conventional_mining'},
                    {'label': 'Artisinal Mining', 'value': 'artisinal_mining'},
                    {'label': 'Blooming', 'value': 'blooming'},
                    {'label': 'Bare Ground', 'value': 'bare_ground'},
                    {'label': 'Water', 'value': 'water'},
                    {'label': 'Road', 'value': 'road'},
                    {'label': 'Agriculture', 'value': 'agriculture'},
                    {'label': 'Habitation', 'value': 'habitation'},
                ],
                value=['partly_cloudy', 'cloudy', 'haze', 'clear', 'slash_and_burn'
                       'selective_logging', 'conventional_mining', 'artisinal_mining', 'blooming',
                       'bare_ground', 'water', 'road', 'agriculture', 'habitation'],
                multi=True
            ),
            ],
            className='six columns',
            style={'width':'50%', 'display':'inline-block'}
            ),

        html.H6 ("file"),
        html.Div([id='waitfor'),
            dcc.Upload(
                id='upload-the-data',
                children=html.Div(
                ["Drag and drop or click to select a file to upload."]
            ),
            style={
                "width": "50%",
                "height": "50px",
                "lineHeight": "50px",
                "borderWidth": "1px",
                "borderStyle": "dashed",
                "boderRadius": "5px",
                "textAlign": "center",
                "margin": "10px",
            },
            multiple=True,
        ),
        html.Div(id='output'),

# using the CSS layout cheat sheet - I'm attempting the "float": "right" formatting.
        html.Div([
            dcc.Graph(id='map', figure=fig,
            style={'width': '50%', 'float':'right'}
            ),
        ])

if __name__ == '__main__':
    app.run_server()
    
''''''
''''''

SyntaxError: invalid syntax (<ipython-input-3-bd92230f03f0>, line 104)

# Resource

### Plotly Dash Tutorial - Working with table and map (Video 04) - YOUTUBE - https://www.youtube.com/watch?time_continue=1050&v=lu0PtsMor4E&feature=emb_logo

### UDEMY COURSE - https://bah.udemy.com/course/interactive-python-dashboards-with-plotly-and-dash/learn/lecture/9570036#overview